A proof of concept showing using a genetic algorithm with our environment.
It is similar to https://github.com/DEAP/deap/blob/a0b78956e28387785e3bb6e2b4b1f1b32c2b3883/examples/ga/onemax_short.py

In [ ]:
# Run this cell if you're using colab. Otherwise, skip it.

!git clone https://github.com/platers/meta-transfer-learning.git

import os
os.chdir('meta-transfer-learning')

!pip install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-0.9.0.dev0-cp36-cp36m-manylinux1_x86_64.whl

!pip install -r requirements.txt

In [1]:
import array
import random

import numpy as np
from typing import Dict

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import gym

from environments import SimpleEnv
import importlib
importlib.reload(SimpleEnv)
from environments.SimpleEnv import SimpleEnv #, TODO: add more environments

from ray.rllib.agents import ppo
from ray import tune
from ray.rllib.policy.policy import Policy
from ray.rllib.env import BaseEnv
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
from ray.rllib.agents.callbacks import DefaultCallbacks
import ray

/home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ray.shutdown()
ray.init()

n_agents = 3
n_var = 2
training_envs = [(SimpleEnv, {
    'n_agents': n_agents,
    'n_vars': n_var,
    'max_step_count': 20,
})]
test_env = (SimpleEnv, {
    'n_agents': n_agents,
    'n_vars': n_var,
    'max_step_count': 20,
})

creator.create('FitnessMax', base.Fitness, weights=(1.0, ))
creator.create('Individual', array.array, typecode='d',
               fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register('attr', random.uniform, -1, 1)
toolbox.register('individual', tools.initRepeat, creator.Individual,
                 toolbox.attr, n_agents * n_var)
toolbox.register('population', tools.initRepeat, list,
                 toolbox.individual)


2020-05-04 19:16:13,525	INFO resource_spec.py:212 -- Starting Ray with 2.54 GiB memory available for workers and up to 1.29 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-05-04 19:16:13,898	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


In [3]:
def evaluate_individual_env(individual, environment, env_config):
    """Runs the environment. All agents have the same policy.
  It returns the total true reward as the fitness.
  """
    #Select random individuals from pop and create the reward weights
    pop = np.array([individual for i in range(n_agents)])
    reward_weights = pop
    #print(individual, reward_weights)
    env_config['reward_weights'] = reward_weights
    #env is only to get action space and observation space
    env = environment(config=env_config)
    class MyCallbacks(DefaultCallbacks):
        #Callback functions to keep track of true reward while training
        def on_episode_start(self, worker: RolloutWorker, base_env: BaseEnv,
                         policies: Dict[str, Policy],
                         episode: MultiAgentEpisode, **kwargs):
            episode.user_data["true_rewards"] = np.zeros(n_agents)

        def on_episode_step(self, worker: RolloutWorker, base_env: BaseEnv,
                        episode: MultiAgentEpisode, **kwargs):
            env = base_env
            true_reward = env.env_states[0].env.last_true_reward
            episode.user_data["true_rewards"] += true_reward

        def on_episode_end(self, worker: RolloutWorker, base_env: BaseEnv,
                       policies: Dict[str, Policy], episode: MultiAgentEpisode,
                       **kwargs):
            true_reward = episode.user_data["true_rewards"]
            for i, r in enumerate(true_reward):
                episode.custom_metrics["true_reward_agent_" + str(i)] = r
            
    config={
        "multiagent": {
            "policies": {
            },
            "policy_mapping_fn":  #all agents share a policy
                lambda agent_id:
                    'agent'
        },
        'env_config': env_config,
        "callbacks": MyCallbacks,
    }
    config['multiagent']['policies']['agent'] = (None, env.observation_space, env.action_space, {})
    trainer = ppo.PPOTrainer(env=environment, config=config)
    
    true_reward_mean = 0
    for i in range(10):
        #print('TRAINING', i)
        true_reward_mean = 0
        custom_metrics = trainer.train()['custom_metrics']  # distributed training step
        #print(custom_metrics)
        for i in range(n_agents):
            true_reward_mean += custom_metrics['true_reward_agent_' + str(i) + '_mean']
    true_reward_mean /= n_agents
    #print('true reward', trainer.collect_metrics()['custom_metrics']['true_reward_mean'])
    print('Evaluated', individual, 'Fitness', true_reward_mean)
    return true_reward_mean


def evaluate_individual(individual):
    """Runs all environments. 
  returns the average true reward over all environments as the fitness.
  """
    fitness = 0
    for env, config in training_envs:
        fitness += evaluate_individual_env(individual, env, config)
    return (fitness, )
    
toolbox.register('evaluate', evaluate_individual)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutFlipBit, indpb=0.05)
toolbox.register('select', tools.selTournament, tournsize=3)

In [ ]:
pop = toolbox.population(n=3)
hof = tools.HallOfFame(10)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register('avg', np.mean)
stats.register('std', np.std)
stats.register('min', np.min)
stats.register('max', np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=3, 
                                   stats=stats, halloffame=hof, verbose=True)

print ('pop', pop)

2020-05-04 19:16:14,812	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-05-04 19:16:14,934	ERROR syncer.py:39 -- Log sync requires rsync to be installed.
2020-05-04 19:16:14,941	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(pid=18945) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=18945)   from ._conv import register_converters as _register_converters
(pid=18944) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=18944)   from ._conv import register_converters as _register_converters


2020-05-04 19:16:24,129	INFO trainable.py:217 -- Getting current IP.
2020-05-04 19:16:24,129	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020-05-04 19:20:02,937	ERROR syncer.py:39 -- Log sync requires rsync to be installed.


Evaluated array('d', [-0.7809296114190265, -0.3162295926872183, 0.8804039548963456, -0.9110896525697714, 0.582323916736595, -0.2089458281705452]) Fitness 18.504254030637767
(pid=18943) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=18943)   from ._conv import register_converters as _register_converters
(pid=18946) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=18946)   from ._conv import register_converters as _register_converters


2020-05-04 19:20:10,992	INFO trainable.py:217 -- Getting current IP.
2020-05-04 19:20:10,995	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020-05-04 19:23:48,966	ERROR syncer.py:39 -- Log sync requires rsync to be installed.


...

2020-05-04 19:23:57,466	INFO trainable.py:217 -- Getting current IP.
2020-05-04 19:23:57,470	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020-05-04 19:27:40,930	ERROR syncer.py:39 -- Log sync requires rsync to be installed.


Evaluated array('d', [-0.8123258034521932, -0.588649071285352, -0.029361298526450863, 0.14840465326327656, -0.8290937720667428, -0.6923058316154422]) Fitness -3.650540992602449
gen	nevals	avg    	std    	min     	max    
0  	3     	8.20373	9.11114	-3.65054	18.5043
(pid=19372) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=19372)   from ._conv import register_converters as _register_converters
(pid=19495) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=19495)   from ._conv import register_converters as _register_converters


2020-05-04 19:27:49,316	INFO trainable.py:217 -- Getting current IP.
2020-05-04 19:27:49,319	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020-05-04 19:30:54,610	ERROR syncer.py:39 -- Log sync requires rsync to be installed.


Evaluated array('d', [-0.7809296114190265, -0.3162295926872183, 0.8804039548963456, -0.9110896525697714, 0.582323916736595, -0.2089458281705452]) Fitness 7.828378641099239
(pid=19584) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=19584)   from ._conv import register_converters as _register_converters
(pid=19587) /home/victor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
(pid=19587)   from ._conv import register_converters as _register_converters


2020-05-04 19:31:03,304	INFO trainable.py:217 -- Getting current IP.
2020-05-04 19:31:03,306	WARNING util.py:37 -- Install gputil for GPU system monitoring.


In [ ]:
best_individual = hof[0]
print(best_individual)

test_reward = evaluate_individual_env(best_individual, test_env[0], test_env[1])
print(test_reward)

In [ ]:
print(evaluate_individual([0, 0, 1, 0, 1, 0])) #Ideal reward, altruistic agent

In [ ]:
hof[:]

In [ ]:
print(evaluate_individual([1, 0, -1, 0, -1, 0])) #Worst reward, selfish agent